<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/hyougo/hyougo_alert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
url = "https://web.pref.hyogo.lg.jp/index.html"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko"
}

In [4]:
alert_dic = {
    "感染小康期": {"判断基準": "10人未満", "対応の方向性": "予防"},
    "感染警戒期": {"判断基準": "10人以上", "対応の方向性": "警戒"},
    "感染増加期": {"判断基準": "20人以上", "対応の方向性": "制限強化"},
    "感染拡大期Ⅰ": {"判断基準": "30人以上", "対応の方向性": "制限強化"},
    "感染拡大期Ⅱ": {"判断基準": "40人以上", "対応の方向性": "制限強化"},
    "感染拡大特別期": {"判断基準": "総合的に判断", "対応の方向性": "制限強化"},
}

In [5]:
r = requests.get(url, headers=headers)
r.raise_for_status()

soup = BeautifulSoup(r.content, "html.parser")

In [6]:
p = re.compile("【現在は、(感染小康期|感染警戒期|感染増加期|感染拡大期[ⅠⅡ]|感染拡大特別期)です】")
# p = re.compile("【現在は、(.+)です】")

In [7]:
tag = soup.find("strong", text=p)
text = ""

if tag:

    s = tag.get_text(strip=True)

    m = re.search(p, s)

    text = m.group(1)

    if m:
        data = alert_dic.get(text)
        print(data["判断基準"], data["対応の方向性"])
    else:
        print("マッチしません")

総合的に判断 制限強化


In [8]:
import pandas as pd
import datetime

In [9]:
JST = datetime.timezone(datetime.timedelta(hours=+9))
dt_now = datetime.datetime.now(JST)

In [10]:
def str2date(s):

    n = re.findall("[0-9]{1,2}", s)

    y = dt_now.year

    if len(n) == 2:
        m, d = map(int, n)
        return pd.Timestamp(y, m, d)

    else:
        return pd.NaT

In [11]:
df = pd.read_html(url, match="新規陽性者数（1週間平均）", index_col=0)[0].T

In [12]:
dt_text = df.columns[0]

In [13]:
df.set_axis(["日付", "新規陽性者数"], axis=1, inplace=True)

In [14]:
df["日付"] = df["日付"].apply(str2date)

In [15]:
df["新規陽性者数"] = df["新規陽性者数"].astype(float)

In [16]:
df.set_index("日付", inplace=True)

In [17]:
df["警戒レベル"] = pd.cut(
    df["新規陽性者数"],
    [0, 10, 20, 30, 40, 100, 9999],
    labels=["感染小康期", "感染警戒期", "感染増加期", "感染拡大期Ⅰ", "感染拡大期Ⅱ", "感染拡大特別期"],
    right=False,
)

In [18]:
df["対応"] = pd.cut(
    df["新規陽性者数"],
    [0, 10, 20, 9999],
    labels=["予防", "警戒", "制限強化"],
    right=False,
)

In [19]:
df

,新規陽性者数,警戒レベル,対応
日付,,,
2020-11-19,86.9,感染拡大期Ⅱ,制限強化
2020-11-20,95.4,感染拡大期Ⅱ,制限強化
2020-11-21,105.9,感染拡大特別期,制限強化
2020-11-22,114.4,感染拡大特別期,制限強化
2020-11-23,119.7,感染拡大特別期,制限強化
2020-11-24,115.6,感染拡大特別期,制限強化
2020-11-25,115.1,感染拡大特別期,制限強化


In [20]:
if text == df.iloc[-1]["警戒レベル"]:
    print(text)
else:
    print("※要確認", text, df.iloc[-1]["警戒レベル"])

感染拡大特別期


In [21]:
dt_text

'(11月25日14時現在）'

In [22]:
month, day, hour = map(int, re.findall("\d{1,2}", dt_text))

In [23]:
dt_update = datetime.datetime(dt_now.year, month, day, hour)

In [24]:
dt_update

datetime.datetime(2020, 11, 25, 14, 0)